![logo](./img/LogoLine_horizon_C3S.png)

## Tutorial on fetching CARRA data from CDS

CARRA data can be found in the CDS (Copernicus Climate Data Store, https://cds.climate.copernicus.eu/cdsapp#!/home).
You need to create an account and accept the terms and conditions for all datasets that you want to use.


### Fetching CARRA data
The source code will download the month of June 2023. By specifying the start and end date below you can easily download longer periods. We are selecting 2m temperature and single levels. 
Here, we download the 3-hourly analysis time steps and the data will be stored in a specified file.
ERA5 data will also be downloaded, for later comparison with CARRA in the next tutorials.
For more details on ERA5 data download please see the [CDS tutorials in this link](https://github.com/ecmwf-projects/copernicus-training-c3s)


In [1]:
# Here one specifies the domain, year and period to download

yearStart = 2023 #first year to download
yearEnd   = 2023 #last year to download
monthStart =  6 # first month to download
monthEnd   =  6 #last month to download
days = [str(i).zfill(2) for i in range(1,31)] # the list of days to download. Adapt day number for other months
domain = 'west_domain' # the domain to select. Use "east_domain" for the East domain

# the local directory where the data will be stored
DATADIR="CARRA"


---

In [4]:
import cdsapi
import os
# create directory to store CARRA data if it does not exist already:
if not os.path.isdir(DATADIR):
    os.makedirs(os.path.join(DATADIR,"Raw_data"))
c = cdsapi.Client()

# define a helper function to download the data
# Note that the time is hardcoded since we are selecting
# all possible available times. 
def fetch_one_month(month, year, days, domain):
    c.retrieve(
        'reanalysis-carra-single-levels',
        {
            'format': 'grib',
            'domain': domain ,
            'level_type': 'surface_or_atmosphere',
            'variable': '2m_temperature',
            'product_type': 'analysis',
            'year': year,
            'month': month,
            'day': days,
            'time': [
                '00:00', '03:00', '06:00',
                '09:00', '12:00', '15:00',
                '18:00', '21:00',
            ],
        },
        os.path.join(DATADIR,"Raw_data",'T2m_an_'+str(year)+"{:02d}".format(month)+'.grb'))

    



In [5]:
# Loop through the period selected
for year in list(range(yearStart, yearEnd + 1)):
    for month in list(range(1,13)):
        if year == yearStart and month < monthStart:
            # Before the start date.
            continue
        if year == yearEnd and month > monthEnd:
            # After end date.
            break

        print ("Data will be fetched for ", year, month)
        print("This will take a few minutes, depending on the data size (~621 MB in this case)")
        fetch_one_month(month, year, days, domain)

Data will be fetched for  2023 6


2024-06-03 09:20:50,670 INFO Welcome to the CDS
2024-06-03 09:20:50,671 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-carra-single-levels
2024-06-03 09:20:50,964 INFO Request is queued
2024-06-03 09:20:52,016 INFO Request is running
2024-06-03 09:23:42,665 INFO Request is completed
2024-06-03 09:23:42,666 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data6/adaptor.mars.external-1717399370.450206-25825-19-d53067bf-e545-4820-aad0-5767a2eb6ee3.grib to CARRA/Raw_data/T2m_an_202306.grb (621.2M)
2024-06-03 09:25:33,262 INFO Download rate 5.6M/s                                                                                            


### Fetching ERA5 data

Here, you downlaod one month of ERA5 data (June 2023). Same as for CARRA, we download 3-hourly data though ERA5 does offer hourly analyses.

---

In [5]:
DATADIR=os.path.join("CARRA","ERA5") 
# create the directory where the data will be stored if it does not exist already

if not os.path.isdir(DATADIR):
    os.makedirs(DATADIR)
# Retrieve the data. Note that the are corresponding roughly to Greenland
# is selected, otherwise all the data for the world will be downloaded.

c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'variable': '2m_temperature',
        'year': str(yearStart),
        'month': str(monthStart).zfill(2),
        'day': days,
        'time': [
            '00:00', '03:00', '06:00',
            '09:00', '12:00', '15:00',
            '18:00', '21:00',
        ],
        'area': [
            90, -110, 55,
            40,
        ],
        'format': 'netcdf',
    },
    os.path.join(DATADIR,'era5_t2m_202306.nc'))

2024-05-15 12:27:20,900 INFO Welcome to the CDS
2024-05-15 12:27:20,901 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-05-15 12:27:21,525 INFO Request is queued
2024-05-15 12:27:22,593 INFO Request is running
2024-05-15 12:28:11,317 INFO Request is completed
2024-05-15 12:28:11,318 INFO Downloading https://download-0001-clone.copernicus-climate.eu/cache-compute-0001/cache/data3/adaptor.mars.internal-1715768883.9647436-16931-13-032aa7aa-fae9-49f8-845a-43b7d22902f6.nc to CARRA/ERA5/era5_t2m_202306.nc (38.8M)
2024-05-15 12:28:17,870 INFO Download rate 5.9M/s                               


Result(content_length=40680676,content_type=application/x-netcdf,location=https://download-0001-clone.copernicus-climate.eu/cache-compute-0001/cache/data3/adaptor.mars.internal-1715768883.9647436-16931-13-032aa7aa-fae9-49f8-845a-43b7d22902f6.nc)